In [8]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.chains import ConversationChain
from typing import Optional, List, Any
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import requests


In [9]:
embedder = SentenceTransformer("bhavyagiri/InLegal-Sbert")

In [10]:
def load_index_and_meta(index_path, meta_path):
    index = faiss.read_index(index_path)
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    return index, meta

def search_faiss(query, index, meta, top_k=2):
    query_vec = embedder.encode([query])
    query_vec = np.array(query_vec).astype("float32")
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        if idx < len(meta):
            results.append(meta[idx])
    return results
class ollama(LLM):
    model_name: str = "mistral"

    @property
    def _llm_type(self) -> str:
        return "ollama"
    
    def _call(self,prompt: str, stop: Optional[List[str]] = None, **kwargs: Any):
        response = requests.post(
        "http://127.0.0.1:11434/api/generate",
        json={"model": self.model_name, "prompt": prompt, "stream": False}
    )
        return response.json()["response"]

In [11]:
ipc_index, ipc_meta = load_index_and_meta("ipc.index", "ipc.json")
judg_index, judg_meta = load_index_and_meta("judgments.index", "judgments.json")

In [12]:
custom=PromptTemplate(input_variables=["history","input"],template=""" You are an legal Chatbot assistant specializing in India Penal Code.
                      
                      Previous conversations:{history}
                      
                      Current question and also judgements :{input}
                      
                    provide an suitable reply for the current question by utilizing previous conversations and keep it around 20 to 30 words .
                      Utilize similar case judgement and compare it with the user case summary and then reply but do not repeat about judgements in every replies.
                      Only use the given text as knowledge, Do Not Retrieve your own knowledge
                      Only Reply using the given IPC sections, STICK TO ONLY THIS TEXT AND PREVIOUS CONVERSATION FOR KNOWLEDGE.
                      Consider youself as an legal advisor and you are the professional.""")

In [13]:
llm=ollama()
memory=ConversationBufferMemory()
conversion = ConversationChain(llm=llm,memory=memory,prompt=custom,verbose=False)

In [14]:
print("⚖️ IPC + Judgement Chatbot (type 'exit' to quit)\n")

user_input = input("You: ").strip()


ipc_results = search_faiss(user_input, ipc_index, ipc_meta)
judg_results = search_faiss(user_input, judg_index, judg_meta)

        # Step 2: Build context
context = "Relevant IPC Sections:\n"
for sec in ipc_results:
    context += f"- Section {sec['section_no']}: {sec['Description']} (Punishment: {sec['punishment_raw']})\n"

prompt = f"User Query: {user_input}\n\n{context}\n\nBased on the above IPC sections, Name the sections relating to the user query and also list possible punishments."
Reply_1=llm._call(prompt)

context += "\nRelevant Judgements:\n"
for case in judg_results:
    context += f"""- Case facts: {case['case_facts']}
Issues: {case['issues']}
Arguments: {case['arguments']}
Decision: {case['decision']}
"""



print(f"\nBot: {Reply_1}\n")

    


⚖️ IPC + Judgement Chatbot (type 'exit' to quit)


Bot:  Based on the user's query, it appears that the act described falls under Section 375 of the Indian Penal Code (IPC), which pertains to rape. The specific sub-section would be:

1. Rape - Against her will.
2. Rape - Without her consent.

As for punishments, the Criminal Law (Amendment) Act, 2013 states that a man found guilty of rape may be punished with imprisonment ranging from 7 years up to life, and shall also be liable to fine. In certain circumstances such as when the victim is a child or the act was particularly brutal or cruel, the punishment may extend to death (capital punishment).

It's important to note that this information provided is for educational purposes only and should not be used as legal advice. If you find yourself in a similar situation, it's crucial to consult with a legal professional for proper guidance.



In [15]:
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    user_input+=f"\n\n similar case judgements : {context}"

    answer = conversion.predict(input=user_input)

    print(f"\nBot: {answer}\n")

Bot: Goodbye!
